In [ ]:
import pandas as pd

In [ ]:
url = "https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_State.csv"
df = pd.read_csv(url)
df.to_csv("data1.csv" , index=False)

In [8]:

import mlflow
from pathlib import Path

def get_project_root():
    return Path().resolve().parent.parent

mlflow.set_tracking_uri(uri=(get_project_root() / 'ETL' / 'dags' / 'mlruns').as_uri())
experiment = mlflow.set_experiment(experiment_name="RealEstate_forcasting")

2025/09/17 01:50:44 INFO mlflow.tracking.fluent: Experiment with name 'RealEstate_forcasting' does not exist. Creating a new experiment.


In [ ]:
from supabase import create_client
import os

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")

df = pd.read_csv("data1.csv")

    # Preprocess data
from preprocessing_1 import preprocess_data_1
df = preprocess_data_1(df)

    # Push preprocessed data to Supabase (state_market table)
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
# Check if data already exists (query row count)
response = supabase.table("state_market").select("*", count="exact").execute()
if response.count == 0:
    data_to_insert = df.to_dict('records')
    supabase.table("state_market").insert(data_to_insert).execute()
else:
    print("Data already exists in Supabase; skipping insertion.")

    # Load full dataset from Supabase
from get_supabase_data import get_supabase_data
df = get_supabase_data() 

df.to_csv("data1.csv", index=False)

In [ ]:
from get_supabase_data import get_supabase_data
df = get_supabase_data() 

In [ ]:
def train_model():
    """Load dataset, train model, save predictions to Supabase"""

    # List of features to predict
    features = [
        "median_listing_price",
        "average_listing_price",
        "median_listing_price_per_square_foot",
        "total_listing_count",
        "median_days_on_market"
    ]

    from preprocessing_2 import preprocess_data_2
    from model_trainer import get_predictions

    target_df = preprocess_data_2(df.copy())
    prediction_df = target_df.copy()

    for feature in features:
        predictions = get_predictions(df, feature)
        prediction_df[feature] = predictions

    for col in features:
        if col in prediction_df.columns:
            prediction_df[col] = prediction_df[col].astype(int)
            
    # Concatenate all predictions
    return prediction_df

In [ ]:
df2 = train_model()